<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Programu Operacyjnego Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Logistic regression

In this exercise you will train a logistic regression model via gradient descent in two simple scenarios.
 
The general setup is as follows:
* we are given a set of pairs $(x, y)$, where $x \in R^D$ is a vector of real numbers representing the features, and $y \in \{0,1\}$ is the target,
* for a given $x$ we model the probability of $y=1$ by $h(x):=g(w^Tx)$, where $g$ is the sigmoid function: $g(z) = \frac{1}{1+e^{-z}}$,
* to find the right $w$ we will optimize the so called logarithmic loss: $J(w) = -\frac{1}{n}\sum_{i=1}^n y_i \log{h(x_i)} + (1-y_i) \log{(1-h(x_i))}$,
* with the loss function in hand we can improve our guesses iteratively:
    * $w_j^{t+1} = w_j^t - \text{step_size} \cdot \frac{\partial J(w)}{\partial w_j}$,
* we can end the process after some predefined number of epochs (or when the changes are no longer meaningful).

Let's start with the simplest example - linear separated points on a plane. 

In [1]:
import numpy as np

np.random.seed(123)

# these parametrize the line
a = 0.3
b = -0.2
c = 0.001

# True/False mapping
def lin_rule(x, noise=0.):
    return a * x[0] + b * x[1] + c + noise < 0.

# Just for plotting
def get_y_fun(a, b, c):
    def y(x):
        return - x * a / b - c / b
    return y

lin_fun = get_y_fun(a, b, c)

In [2]:
# Training data

n = 500
range_points = 1
sigma = 0.05

X = range_points * 2 * (np.random.rand(n, 2) - 0.5)
y = [lin_rule(x, sigma * np.random.normal()) for x in X]

print(X[:10])
print(y[:10])

[[ 0.39293837 -0.42772133]
 [-0.54629709  0.10262954]
 [ 0.43893794 -0.15378708]
 [ 0.9615284   0.36965948]
 [-0.0381362  -0.21576496]
 [-0.31364397  0.45809941]
 [-0.12285551 -0.88064421]
 [-0.20391149  0.47599081]
 [-0.63501654 -0.64909649]
 [ 0.06310275  0.06365517]]
[False, True, False, False, False, True, False, True, True, False]


Let's plot the data.

In [3]:
import plotly.express as px

# plotly has a problem with coloring boolean values, hence stringify
# see https://community.plotly.com/t/plotly-express-scatter-color-not-showing/25962
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))
x_range = [np.min(X[:, 0]), np.max(X[:, 1])]
fig.add_scatter(x=x_range, y=list(map(lin_fun, x_range)), name='ground truth border')
fig.show()

Now, let's implement and train a logistic regression model. You should obtain accuracy of at least 96%.

In [5]:
Xn = np.array(X)
yn = np.array(y)
yn = yn.reshape(-1,1)

def sigmoid(z):
    return 1 / (1+np.exp(-z))

def loss(y, yh):
    loss_without_mean = y * np.log(yh) - (1-y) * np.log(1-yh)
    return loss_without_mean.mean()

def gradient_w(X, y, yh):
    return (1 / X.shape[0]) * (X.T @ (yh - y))

def gradient_b(y, yh):
    return np.mean(yh-y)

def accuracy(X, y, w, b):
    probabilities = sigmoid(np.dot(X, w)+b)
    predicts = np.array([1 if i >= 1/2 else 0 for i in probabilities])
    return np.mean(y == predicts)

def training(X, y):
    epochs_nr = 40
    step = 1

    w = np.zeros((2,1))
    b = 1

    losses = []

    for _ in range(epochs_nr):
        yh = sigmoid((X @ w) + b)

        dw = gradient_w(X, y, yh)
        db = gradient_b(y, yh)

        w -= step * dw
        b -= step * db

        yh = sigmoid((X @ w) + b)
        l = loss(y, yh)
        losses.append(l)

    return w, b, losses

w, b, losses = training(Xn, yn)
acc = accuracy(Xn, yn.flatten(), w, b)

print(acc)
print(w, b)

0.968
[[-3.19667776]
 [ 2.13984449]] 0.06298916889905873


Let's visually asses our model. We can do this by using our estimates for $a,b,c$.

In [6]:
#################################################################
# TODO: Pass your estimates for a,b,c to the get_y_fun function #
#################################################################
lin_fun2 = get_y_fun(w[0][0], w[1][0], b)

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))
x_range = [np.min(X[:, 0]), np.max(X[:, 1])]
fig.add_scatter(x=x_range, y=list(map(lin_fun, x_range)), name='ground truth border')
fig.add_scatter(x=x_range, y=list(map(lin_fun2, x_range)), name='estimated border')
fig.show()

Let's now complicate the things a little bit and make our next problem nonlinear.

In [7]:
# Parameters of the ellipse
s1 = 1.
s2 = 2.
r = 0.75
m1 = 0.15
m2 = 0.125

# 0/1 mapping, checks whether we are inside the ellipse
def circle_rule(x, y, noise=0.):
    return 1 if s1 * (x - m1) ** 2 + s2 * (y - m2) ** 2 + noise < r ** 2 else 0

In [8]:
# Training data

n = 500
range_points = 1

sigma = 0.1

X = range_points * 2 * (np.random.rand(n, 2) - 0.5)

y = [circle_rule(x, y, sigma * np.random.normal()) for x, y in X]

print(X[:10])
print(y[:10])

[[ 0.18633789  0.87560968]
 [-0.81999293  0.61838609]
 [ 0.22604784  0.28001611]
 [ 0.9846182  -0.35783437]
 [-0.27962406  0.07170775]
 [ 0.2501677  -0.37650776]
 [ 0.41264707 -0.8357508 ]
 [-0.61039043 -0.97349628]
 [ 0.49924022  0.89579621]
 [ 0.537422   -0.65425777]]
[0, 0, 1, 0, 1, 1, 0, 0, 0, 0]


Let's plot the data.

In [9]:
import plotly.graph_objects as go

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))

xgrid = np.arange(np.min(X[:, 0]), np.max(X[:, 0]), 0.003)
ygrid = np.arange(np.min(X[:, 1]), np.max(X[:, 1]), 0.003)
contour =  go.Contour(
        z=np.vectorize(circle_rule)(*np.meshgrid(xgrid, ygrid, indexing="ij")),
        x=xgrid, 
        y=ygrid
    )
fig.add_trace(contour)
fig.show()

Now, let's train a logistic regression model to tackle this problem. Note that we now need a nonlinear decision boundary. You should obtain accuracy of at least 90%.

Hint: 
<sub><sup><sub><sup><sub><sup>
Use feature engineering.
</sup></sub></sup></sub></sup></sub>

In [15]:
Xn = np.array(X)
Xn = np.c_[Xn, Xn ** 2]
yn = np.array(y)
yn = yn.reshape(-1,1)

def sigmoid(z):
    return 1 / (1+np.exp(-z))

def loss(y, yh):
    loss_without_mean = y * np.log(yh) - (1-y) * np.log(1-yh)
    return loss_without_mean.mean()

def gradient_w(X, y, yh):
    return (1 / X.shape[0]) * (X.T @ (yh - y))

def gradient_b(y, yh):
    return np.mean(yh-y)

def accuracy(X, y, w, b):
    probabilities = sigmoid(np.dot(X, w)+b)
    predicts = np.array([1 if i >= 1/2 else 0 for i in probabilities])
    return np.mean(y == predicts)

def training(X, y):
    epochs_nr = 500
    step = 1

    w = np.zeros((4,1))
    b = 1

    losses = []

    for _ in range(epochs_nr):
        yh = sigmoid((X @ w) + b)

        dw = gradient_w(X, y, yh)
        db = gradient_b(y, yh)

        w -= step * dw
        b -= step * db

        yh = sigmoid((X @ w) + b)
        l = loss(y, yh)
        losses.append(l)

    return w, b, losses

w, b, losses = training(Xn, yn)
acc = accuracy(Xn, yn.flatten(), w, b)

print(acc)
print(w, b)

0.954
[[ 1.63865281]
 [ 2.63294292]
 [-5.36684495]
 [-9.84147322]] 2.5146214632066273


Let's visually asses our model. 

Contrary to the previous scenario, converting our weights to parameters of the ground truth curve may not be straightforward. It's easier to just provide predictions for a set of points in $R^2$.

In [16]:
h = .02

xgrid = np.arange(np.min(X[:, 0]), np.max(X[:, 0]), h)
ygrid = np.arange(np.min(X[:, 1]), np.max(X[:, 1]), h)

xx, yy = np.meshgrid(xgrid, ygrid, indexing="ij")
X_plot = np.c_[xx.ravel(), yy.ravel()]

_X = np.concatenate([X_plot, X_plot**2], axis=1)
_w = w
_b = b

def logistic_regression(w, b, X):
    return sigmoid((X @ w) + b)

preds = logistic_regression(_w, _b, _X)
preds = preds.flatten()

print(preds.shape)

(10000,)


In [17]:
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))

xx, yy = np.meshgrid(xgrid, ygrid, indexing="ij")

contour = go.Contour(z=preds.reshape(len(xgrid), len(ygrid)), x=xgrid, y=ygrid)
fig.add_trace(contour)
fig.show()

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>